In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

import spacy

import ssl
import socket

In [2]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)

In [3]:
mails = pd.read_csv('merged_datasets.csv')
categories = pd.CategoricalDtype(['safe', 'phishing'], ordered=True)
mails['label'].astype(categories)

0        phishing
1        phishing
2        phishing
3            safe
4        phishing
5        phishing
6        phishing
7        phishing
8            safe
9        phishing
10       phishing
11       phishing
12       phishing
13       phishing
14       phishing
15           safe
16       phishing
17       phishing
18           safe
19           safe
20           safe
21           safe
22           safe
23           safe
24           safe
25       phishing
26       phishing
27           safe
28       phishing
29       phishing
30       phishing
31       phishing
32       phishing
33           safe
34           safe
35       phishing
36       phishing
37       phishing
38           safe
39       phishing
40       phishing
41       phishing
42       phishing
43           safe
44           safe
45           safe
46       phishing
47           safe
48       phishing
49       phishing
50           safe
51           safe
52           safe
53           safe
54           safe
55        

In [4]:
mails.drop_duplicates(subset=['subject', 'sender_mail'],
                      keep='first',
                      inplace=True)


mails.dropna(subset=['body', 'sender_mail'], inplace=True)

In [5]:
# # Shufflujemy sobie
# mails = mails.sample(frac=1).reset_index(drop=True)

# # Na potrzeby obliczeń zmniejszamy
# cut_rows_to = 25000
# mails = mails.iloc[:cut_rows_to, :]
# print(mails.phishing.value_counts())

In [6]:
def extract_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    matches = re.findall(url_pattern, text)

    return matches, len(matches)

mails[['extracted_urls', 'urls_count']] = mails['body'].apply(lambda x:
                                                              pd.Series(extract_urls(x)))


def map_url_lens(lens):
    if lens <= 2:
        return str(lens)
    else:
        return '3<='

mails['urls_count'] = mails['urls_count'].apply(map_url_lens)
categories = pd.CategoricalDtype(['0', '1', '2', '3<='], ordered=True)
mails['urls_count'] = mails['urls_count'].astype(categories)

In [7]:
# Jeżeli jeden url w mailu jest fałszywy, to wychodzimy z założenia, że inne też
def return_random_url(urls):
    urls_len = len(urls)
    if urls_len == 0:
        return None
    

    random_index = np.random.randint(0, urls_len)
    randomized_url = urls[random_index]
    return randomized_url

mails['in_body_url'] = mails['extracted_urls'].apply(return_random_url)

In [8]:
def is_https(url):
    if url is None:
        return 'None'

    protocol = url[:5].lower()
    return 'https' if protocol == 'https' else 'http'


mails['protocol'] = mails['in_body_url'].apply(is_https)
categories = pd.CategoricalDtype(['None', 'http', 'https'], ordered=True)
mails['protocol'] = mails['protocol'].astype(categories)
legit_rows = mails[mails['label'] == 'safe']
print(legit_rows['protocol'].value_counts(normalize=True, dropna=False))

protocol
None     0.623623
http     0.354750
https    0.021627
Name: proportion, dtype: float64


In [9]:
def get_domain(url):
    if url is None:
        return 'no url'

    # Get domain out of url
    pattern = re.compile(r'https?://([^/]+)')
    match = pattern.match(url)
    
    if match:
        domain = match.group(1)
        return domain
    else:
        return 'url error'


mails['domain_(in_body_url)'] = mails['in_body_url'].apply(get_domain)

In [12]:
print(mails.info(), end='\n\n')
print(mails['label'].value_counts(normalize=True), end='\n\n')
print(mails['protocol'].value_counts(normalize=True, dropna=False), end='\n\n')
print(mails['urls_count'].value_counts(normalize=True, dropna=False), end='\n\n')


label
phishing    0.512992
safe        0.487008
Name: proportion, dtype: float64

protocol
None     0.523969
http     0.463022
https    0.013009
Name: proportion, dtype: float64

urls_count
0      0.523969
1      0.254482
3<=    0.122191
2      0.099358
Name: proportion, dtype: float64

